In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [4]:
import pandas as pd
import random
from matplotlib import pyplot as plt

# Pandas options
pd.set_option('display.max_colwidth', 1000, 'display.max_rows', None, 'display.max_columns', None)

import seaborn as sns
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

In [5]:
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
loan = pd.read_csv('/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz', compression='gzip', low_memory=True)
loan.head()

In [7]:
loan.info()

In [8]:
loans = loan[['loan_amnt', 'term','int_rate', 'sub_grade','emp_title',
                  'emp_length','home_ownership', 'annual_inc', 'loan_status', 'addr_state',
                  'dti','mths_since_recent_inq', 'revol_util', 'bc_open_to_buy', 'bc_util', 'num_op_rev_tl']]

In [9]:
loans.head()

In [10]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [11]:
loans.describe()

# Missing data

In [12]:
loans.isnull().sum()

In [13]:
missing_data = pd.DataFrame({'total_missing': loans.isnull().sum(), '%_missing': 
(loans.isnull().sum()/2260701)*100})
missing_data

In [14]:
loans = loans.dropna()
loans.info()

In [15]:
missing_data = pd.DataFrame({'total_missing': loans.isnull().sum()})
missing_data

# Dealing with outliers

In [16]:
loans.hist(bins = 10, figsize = (20,10), color = 'b')

In [17]:
%matplotlib inline
from matplotlib import pyplot as plt
num_cols = ['loan_amnt', 'term','int_rate', 'sub_grade',
                  'emp_length','home_ownership', 'annual_inc', 'loan_status', 'addr_state',
                  'dti','mths_since_recent_inq', 'revol_util', 'bc_open_to_buy', 'bc_util', 'num_op_rev_tl' ]
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.title("Lending club dataset", fontsize=20)
plt.show()

In [18]:
loans.annual_inc.describe()

In [19]:
loans.annual_inc.unique()

In [20]:
num_cols = ['annual_inc']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.title("Numerical variable", fontsize=20)
plt.show()

In [21]:
#remove outlier "annual income"
q_low = loans["annual_inc"].quantile(0.08)
q_hi  = loans["annual_inc"].quantile(0.92)

loans = loans[(loans["annual_inc"] < q_hi) & (loans["annual_inc"] > q_low)]

In [22]:
# plot a chart to view to what extend outliers are removed
num_cols = ['annual_inc']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.title("Numerical variable", fontsize=20)
plt.show()

In [23]:
num_cols = ['dti']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [24]:
#remove outlier 

loans = loans[(loans['dti'] <=45)]

In [25]:
num_cols = ['dti']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [26]:
loans.hist(bins = 30, figsize = (20,10), color = 'b')

In [27]:
num_cols = ['bc_open_to_buy']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [28]:
#remove outlier 
q_hi  = loans['bc_open_to_buy'].quantile(0.95)
loans = loans[(loans['bc_open_to_buy'] < q_hi)]

In [29]:
num_cols = ['bc_open_to_buy']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [30]:
loans.hist(bins = 30, figsize = (20,10), color = 'b')

In [31]:
num_cols = ['bc_util']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [32]:
#remove outlier 

loans = loans[(loans['bc_util'] <=160)]

In [33]:
num_cols = ['bc_util']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [34]:
num_cols = ['revol_util']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [35]:
#remove outlier 
loans = loans[(loans['revol_util'] <=150)]

In [36]:
num_cols = ['revol_util']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [37]:
num_cols = ['num_op_rev_tl']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [38]:
#remove outlier
loans = loans[(loans['num_op_rev_tl'] <=35)]

In [39]:
num_cols = ['num_op_rev_tl']
plt.figure(figsize=(18,9))
loans[num_cols].boxplot()
plt.show()

In [40]:
loans.hist(bins = 10, figsize = (20,10), color = 'b')

# Panda Profiling

In [41]:
# dropping passed columns
loans.drop(["bc_util", "bc_open_to_buy","int_rate"], axis = 1, inplace = True)
loans.head()

# Categorcal features

In [42]:
cleaner_app_type = {"term": {" 36 months": 1.0, " 60 months": 2.0},
                    "sub_grade": {"A1": 1.0, "A2": 2.0, "A3": 3.0, "A4": 4.0, "A5": 5.0,
                                  "B1": 11.0, "B2": 12.0, "B3": 13.0, "B4": 14.0, "B5": 15.0,
                                  "C1": 21.0, "C2": 22.0, "C3": 23.0, "C4": 24.0, "C5": 25.0,
                                  "D1": 31.0, "D2": 32.0, "D3": 33.0, "D4": 34.0, "D5": 35.0,
                                  "E1": 41.0, "E2": 42.0, "E3": 43.0, "E4": 44.0, "E5": 45.0,
                                  "F1": 51.0, "F2": 52.0, "F3": 53.0, "F4": 54.0, "F5": 55.0,
                                  "G1": 61.0, "G2": 62.0, "G3": 63.0, "G4": 64.0, "G5": 65.0,
                                    },
                     "emp_length": {"< 1 year": 0.0, '1 year': 1.0, '2 years': 2.0, '3 years': 3.0, '4 years': 4.0, 
                                   '5 years': 5.0, '6 years': 6.0, '7 years': 7.0, '8 years': 8.0, '9 years': 9.0,
                                   '10+ years': 10.0 }
                   }
loans = loans.replace(cleaner_app_type)
loans.info()

# Ordinal transofmraiton

In [43]:
loans['loan_status'].value_counts()

In [44]:
array = ['Charged Off', 'Fully Paid']
loans = loans.loc[loans['loan_status'].isin(array)]
loans.head()

In [45]:
cleaner_app_type1 = {"loan_status": { "Fully Paid": 1.0, "Charged Off": 0.0}}
loans = loans.replace(cleaner_app_type1)
loans.info()

In [46]:
loans['loan_status'].value_counts()

In [47]:
plt.subplot(2, 1, 1)
sns.countplot(x='home_ownership', data=loans, hue='loan_status')

In [48]:
analyse_home_ownership = loans.groupby(['home_ownership','loan_status'])['loan_status'].count()
analyse_home_ownership = analyse_home_ownership.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

analyse_home_ownership

In [49]:
loans['emp_title'].nunique()

In [50]:
emp_title_data = loan['emp_title'].value_counts()[:20]

In [51]:
loan_emp_title = loan.loc[loan['loan_status'] =='Fully Paid'] 
loan_emp_title.head()

In [52]:
plt.figure(figsize=(15, 12))
plt.subplot(2, 2, 2)
plt.barh(loan_emp_title.emp_title.value_counts()[:10].index, loan.emp_title.value_counts()[:10])
plt.title("The most 10 jobs title applied for a loan")
plt.tight_layout()

# One hot encoding

In [53]:
#convert annual_inc to integer data type

loans = loans.astype({"home_ownership":'category', "addr_state":'category'})
loans.dtypes

In [54]:
cat_columns = ["home_ownership", "addr_state"]
#create a new DataFrame for our processed data
loans = pd.get_dummies(loans, prefix_sep="__",
                              columns=cat_columns)
loans.head()

In [55]:
pip install hvplot

# Bi-Variate Analysis
'sub_grade' and 'loan_status'

In [56]:
analyse_sub_grade = loans.groupby(['sub_grade','loan_status'])['loan_status'].count()
analyse_sub_grade = analyse_sub_grade.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
analyse_sub_grade

In [57]:
analyse_sub_grade = analyse_sub_grade.unstack()

In [58]:
analyse_sub_grade.plot.area()

'annual_inc' vs 'loan_status'

In [59]:
bins = [30000, 50000, 70000, 90000, 110000, 130000, 150000]
labels = ['30-50k', '50-70k', '70-90k', '90-110k','110-130k','130-150k']
loans['binned'] = pd.cut(loans['annual_inc'], bins=bins, labels=labels)
loans.head()

In [60]:
analyse_income = loans.groupby(['binned','loan_status'])['loan_status'].count()
analyse_income

In [61]:
analyse_income = analyse_income.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

analyse_income

In [62]:
analyse_income = analyse_income.unstack()
analyse_income

In [63]:
analyse_income.plot.area()

'emp_length' vs 'loan_status'

In [64]:
analyse_emp_length = loans.groupby(['emp_length','loan_status'])['loan_status'].count()
analyse_emp_length = analyse_emp_length.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

analyse_emp_length

In [65]:
analyse_emp_length = analyse_emp_length.unstack()
analyse_emp_length.plot.area()

In [66]:
analyse_dti = loans.groupby(['dti','loan_status'])['loan_status'].count()
loans['dti'].unique()

In [67]:
binsdti = [1, 10, 20, 30, 40, 50]
labelsdti = ['1-10', '10-20', '20-30','30-40','40-50']
loans['binneddti'] = pd.cut(loans['dti'], bins=binsdti, labels=labelsdti)


analyse_dti = loans.groupby(['binneddti','loan_status'])['loan_status'].count()
analyse_dti = analyse_dti.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

analyse_dti

In [68]:
analyse_dti = analyse_dti.unstack()
analyse_dti.plot.area()

# Train/Test Split

In [69]:
loans.drop(["emp_title", "binned","binneddti"], axis = 1, inplace = True)

In [70]:
X = loans.drop('loan_status', axis=1)
y = loans[['loan_status']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [71]:
loans.head()

# Logistic Regression Model

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [73]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [74]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

ax = sns.heatmap(confusion_matrix, cmap='viridis_r', annot=True, fmt='d', square=True)
accuracy_score(y_pred , y_test)